# Understanding Goal Programming

Goal Programming (GP) is a branch of multi-objective optimization that focuses on achieving specific targets or goals when it's impossible to optimize all objectives simultaneously due to inherent conflicts or trade-offs. Unlike Multi-objective Decision Making (MODM), which seeks to find a set of Pareto-optimal solutions without explicit prioritization, GP allows decision-makers to specify desired goals and prioritize them, aiming to minimize the deviations from these goals.

In GP, each objective is transformed into a goal, with decision-makers specifying the target levels they wish to achieve. The essence of GP lies in its ability to handle multiple, often competing goals by finding a solution that minimizes the deviations from these target levels. This approach is particularly useful in real-world scenarios where satisfying all objectives to their fullest extent is impractical or impossible.

The methodology involves defining not only the goals but also the weights or priorities associated with each goal, reflecting their relative importance to the decision-maker. This prioritization helps in guiding the optimization process, ensuring that more critical goals are satisfied as closely as possible before considering less critical ones.

GP models are formulated similarly to linear programming models but include additional variables to capture the deviations from each goal. These deviation variables are then minimized in the objective function, subject to the constraints of the problem. This formulation allows for a flexible and nuanced approach to decision-making, accommodating the complex realities and trade-offs that decision-makers face.

In practice, GP is applied across various fields, from resource allocation and project management to financial planning and engineering design. Its versatility and practicality make it a powerful tool for tackling decision-making problems where multiple objectives must be balanced according to their importance.

**Sustainable Campus Cafe Operations Model**

In our scenario, we aim to optimize the operations of a campus cafe with a focus on sustainability. This involves balancing multiple objectives: maximizing profit, minimizing waste, and maximizing customer satisfaction. Here's the mathematical model.

**Decision Variables:**
- $x_{ij}$: Number of units of product $i$ sold during time period $j$, where $i$ represents different products and $j$ represents different time periods throughout the day.

**Parameters:**
- $p_{i}$: Profit per unit of product $i$.
- $w_{ij}$: Waste associated with product $i$ during time period $j$.
- $s_{ij}$: Customer satisfaction score for product $i$ during time period $j$.
- $C_{i}$: Cost of producing one unit of product $i$.
- $B$: Total budget for operational costs.
- $L$: Available labor hours per day.
- $D_{ij}$: Demand for product $i$ during time period $j$.

**Objective Functions:**
1. Maximize Total Profit:
   $$Z_1 = \sum_{i}\sum_{j} p_{i} \cdot x_{ij} - C_{i} \cdot x_{ij}$$
2. Minimize Total Waste:
   $$Z_2 = \sum_{i}\sum_{j} w_{ij} \cdot x_{ij}$$
3. Maximize Customer Satisfaction:
   $$Z_3 = \sum_{i}\sum_{j} s_{ij} \cdot x_{ij}$$

**Constraints:**
1. Budget Constraint:
   $$\sum_{i}\sum_{j} C_{i} \cdot x_{ij} \leq B$$
2. Labor Constraint:
   $$\sum_{i}\sum_{j} \text{Labor required for } x_{ij} \leq L$$
3. Demand Satisfaction:
   $$x_{ij} \geq D_{ij}, \forall i, j$$
4. Non-negativity and Integrality:
   $$x_{ij} \geq 0 \text{ and integer}, \forall i, j$$

In [1]:
import pulp as lp

# Define the problem
model = lp.LpProblem("Sustainable_Campus_Cafe_Operations", lp.LpMinimize)

# Decision Variables
products = ['Coffee', 'Sandwich', 'Salad']  # Example products
time_periods = ['Morning', 'Afternoon']  # Example time periods
x_ij = lp.LpVariable.dicts("UnitsSold", (products, time_periods), lowBound=0, cat='Integer')

# Parameters (Example values)
profit = {'Coffee': 1.5, 'Sandwich': 2.0, 'Salad': 1.8}
cost = {'Coffee': 0.5, 'Sandwich': 1.0, 'Salad': 0.8}
waste = {'Coffee': 0.1, 'Sandwich': 0.2, 'Salad': 0.15}
satisfaction = {'Coffee': 0.9, 'Sandwich': 0.8, 'Salad': 0.85}
demand = {('Coffee', 'Morning'): 50, ('Coffee', 'Afternoon'): 30, 
          ('Sandwich', 'Morning'): 40, ('Sandwich', 'Afternoon'): 50,
          ('Salad', 'Morning'): 30, ('Salad', 'Afternoon'): 20}
B = 1000  # Total budget
L = 100  # Labor hours available

# Objective Functions & Deviation Variables
# For Goal Programming, we introduce deviation variables for each goal
d_plus_profit = lp.LpVariable("d_plus_profit", lowBound=0)  # Deviation from profit goal
d_plus_waste = lp.LpVariable("d_plus_waste", lowBound=0)  # Deviation from waste minimization
d_plus_satisfaction = lp.LpVariable("d_plus_satisfaction", lowBound=0)  # Deviation from satisfaction goal

# Set the objective function for Goal Programming
model += d_plus_profit + d_plus_waste + d_plus_satisfaction

# Constraints
# Profit Goal
model += lp.lpSum([profit[p] * x_ij[p][t] for p in products for t in time_periods]) - d_plus_profit >= 200, "ProfitGoal"

# Waste Minimization Goal
model += lp.lpSum([waste[p] * x_ij[p][t] for p in products for t in time_periods]) + d_plus_waste <= 10, "WasteGoal"

# Customer Satisfaction Goal
model += lp.lpSum([satisfaction[p] * x_ij[p][t] for p in products for t in time_periods]) - d_plus_satisfaction >= 100, "SatisfactionGoal"

# Budget Constraint
model += lp.lpSum([cost[p] * x_ij[p][t] for p in products for t in time_periods]) <= B, "BudgetConstraint"

# Demand Satisfaction and Labor Constraints
for p, t in demand:
    model += x_ij[p][t] >= demand[(p, t)], f"Demand_{p}_{t}"

# Solve the model
model.solve()

# Display results
if lp.LpStatus[model.status] == 'Optimal':
    print("Optimal solution found:")
    for p in products:
        for t in time_periods:
            print(f" - Units of {p} sold during {t}: {x_ij[p][t].value()}")
    print(f"Objective Values: Profit Deviation = {d_plus_profit.value()}, Waste Deviation = {d_plus_waste.value()}, Satisfaction Deviation = {d_plus_satisfaction.value()}")
else:
    print("No optimal solution found.")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/OR/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/910d3f146fe347bc8e3a6e40ad364cb9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/910d3f146fe347bc8e3a6e40ad364cb9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 64 RHS
At line 75 BOUNDS
At line 82 ENDATA
Problem MODEL has 10 rows, 9 columns and 33 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.02

No optimal solution found.
